In [95]:
import numpy as np

In [96]:
M = np.array([[1, 2, 0], [1, 3, 1], [-2, 0, 1]], np.double)
b = np.array([3, 5, -1], np.double).reshape((3, 1))
dim = b.shape[0]

# Exercise 1

In [97]:
def triangular_lower_solve(L, b, dim):
    # Solve Ly = b
    y = np.zeros((dim, 1))
    for i in range(0, dim): # Iterate over each L line
        y[i] = b[i]
        for j in range(0, i): # Iterate over columns until i
            y[i] -= L[i][j]*y[j]
        y[i] /= L[i][i]
    
    return y

L_test = np.array([[1, 0, 0], [2, 1, 0], [3, 2, 1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

triangular_lower_solve(L_test, b_test, 3)

array([[-1.],
       [ 2.],
       [ 5.]])

In [98]:
def triangular_upper_solve(U, y, dim):
    # Solve Ux = y
    x = np.zeros((dim, 1))
    for i in range(dim-1, -1, -1): # Iterate over each U line, reversed
        x[i] = y[i]
        for j in range(dim-1, i, -1): # Iterate over columns until 0, from right to left
            x[i] -= U[i][j]*x[j]
        x[i] /= U[i][i]
    
    return x

U_test = np.array([[2, 1, 2], [0, 1, -1], [0, 0, -5]], np.double)
y_test = np.array([-1, 2, 5], np.double).reshape((3, 1))

triangular_upper_solve(U_test, y_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])

# Exercise 2

In [99]:
def lu_factorization(M, dim):
    L = np.identity(dim)
    U = np.zeros((dim,dim))
    
    U[0] = M[0] # Iterate first line
    
    for it in range(0, dim-1):
        
        for line in range(it+1, dim): # Iterate it'th column
            sum = 0
            for it2 in range(0, it+2):
                sum += L[line][it2]*U[it2][it]
            L[line][it] = (M[line][it] - sum)/U[it][it]
        
        for col in range(it+1, dim): # Iterate (it+1)'th line
            sum = 0
            for it2 in range(0, it+2):
                sum += L[it+1][it2]*U[it2][col]
            U[it+1][col] = M[it+1][col] - sum
        
    return L, U

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
L_test, U_test = lu_factorization(M_test, 3)

print(L_test)
print(U_test)

[[1. 0. 0.]
 [2. 1. 0.]
 [3. 2. 1.]]
[[ 2.  1.  2.]
 [ 0.  1. -1.]
 [ 0.  0. -5.]]


In [100]:
L, U = lu_factorization(M, dim)

# Solve Ly = b
y = triangular_lower_solve(L, b, dim)

# Solve Ux = y
x = triangular_upper_solve(U, y, dim)

x2 = np.linalg.solve(M, b)

print('x \n', x)
print()
print('x2\n', x2)
np.allclose(x, x2)

x 
 [[1.]
 [1.]
 [1.]]

x2
 [[1.]
 [1.]
 [1.]]


True

# Exercise 3

In [101]:
def gauss_elimination(M, b, dim):
    
    U = M.copy()
    y = b.copy()
    
    for col in range(0, dim-1):
        col_value = U[col][col]
        for line in range(col+1, dim):
            multiplier = -U[line][col]/col_value
            U[line] = U[col]*multiplier + U[line]
            y[line] = y[col]*multiplier + y[line]
    
    return triangular_upper_solve(U, y, dim)

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

gauss_elimination(M_test, b_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])

# Exercise 4

In [103]:
def gauss_elimination_with_pivot(M, b, dim):
    
    U = M.copy()
    y = b.copy()
    
    for col in range(0, dim-1):
        
        current_pivot = np.absolute(U[col:,col]).argmax(axis=0)+col
        U[[col, current_pivot]] = U[[current_pivot, col]]
        y[[col, current_pivot]] = y[[current_pivot, col]]
        
        col_value = U[col][col]
        for line in range(col+1, dim):
            multiplier = -U[line][col]/col_value
            U[line] = U[col]*multiplier + U[line]
            y[line] = y[col]*multiplier + y[line]
            
    return triangular_upper_solve(U, y, dim)

M_test = np.array([[2, 1, 2], [4, 3, 3], [6, 5, -1]], np.double)
b_test = np.array([-1, 0, 6], np.double).reshape((3, 1))

gauss_elimination_with_pivot(M_test, b_test, 3)

array([[ 0.],
       [ 1.],
       [-1.]])